In [30]:
import sys
sys.path.append('../')
import numpy as np
import pandas as pd
from grid_world.grid_world import GridWorld
from DMEIRL.DeepMEIRL_FC import DMEIRL
from grid_world.trajGen_grid_world import GridWorld_trajGen
from utils import utils

from grid_world.data_parser import DataParser
from grid_world.grid_world import GridWorld


import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
parser = DataParser(width=10,height=10)
from PIL import Image
import os
from tqdm import tqdm
tracks_path = 'demo_label/train/'
file_names = os.listdir(tracks_path)
#read images
imgs = []
for file_name in file_names:
    imgs.append(Image.open(tracks_path + "/" + file_name))

#parse images
for i in tqdm(range(len(imgs))):
    parser.ParseEnvironmentFromImage(imgs[i],file_names[i].split('.')[0],save_path='demo_env&feature')

100%|██████████| 4/4 [00:00<00:00, 252.96it/s]


In [32]:
n = np.nan
state_mat = np.array([[ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                       [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                       [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                       [ 0, 0, 0, n, n, 0, n, n, n, n],
                       [ 0, 0, 0, n, 0, 0, 0, 0, 0, 0],
                       [ 0, 0, 0, n, 0, 0, 0, 0, 0, 0],
                       [ 0, 0, 0, n, 0, 0, 8, 8, 8, 8],
                       [ 0, 0, 0, n, 0, 0, 8, 8, 8, 8],
                       [ 0, 0, 0, n, 0, 0, 8, 8, 8, 8],
                       [ 0, 0, 0, n, 0, 0, 8, 8, 8, 8],])

world_trajGen = GridWorld_trajGen(10,10,states_matrix=state_mat,
                                  features_folderPath='demo_env&feature/features_grid/0119_10x10',
                                  rewards_mul=[1,-2,-1,-1],
                                  init_states=list(range(0,10)),
                                  trans_prob=0.6,discount=0.9)
world_trajGen.ShowRewards(title='真实奖励')

In [33]:
world_trajGen.GenerateTrajectories(traj_count=500,traj_length=20,save=True)

100%|██████████| 500/500 [00:00<00:00, 4966.73it/s]


,m,trajs
0,1,"[(0, 4, 1), (1, 4, 2), (2, 4, 12), (12, 4, 13)..."
1,2,"[(0, 4, 1), (1, 4, 2), (2, 4, 3), (3, 4, 4), (..."
2,3,"[(5, 1, 15), (15, 1, 25), (25, 1, 25), (25, 1,..."
3,4,"[(0, 4, 10), (10, 1, 20), (20, 1, 20), (20, 1,..."
4,5,"[(2, 4, 3), (3, 4, 4), (4, 4, 5), (5, 1, 6), (..."
...,...,...
495,496,"[(5, 1, 5), (5, 1, 15), (15, 1, 14), (14, 4, 1..."
496,497,"[(3, 4, 4), (4, 4, 5), (5, 1, 15), (15, 1, 25)..."
497,498,"[(0, 4, 1), (1, 4, 2), (2, 4, 12), (12, 4, 2),..."
498,499,"[(8, 3, 9), (9, 0, 9), (9, 0, 9), (9, 0, 9), (..."


In [34]:
world_trajGen.ShowTrajs_Experts()

In [35]:
world = GridWorld(
    environments_img_folderPath='demo_label/train/',
    expert_trajs=world_trajGen.df_trajs_experts,
    width=10,
    height=10,
    active_all=False,
    trans_prob=0.6,
    discount=0.9,
    real_reward_mat = world_trajGen.real_rewards_matrix
)
world.ShowGridWorld_Count_log(title='轨迹计数分布(对数)')

parsing environments from folder:: 100%|██████████| 4/4 [00:00<00:00, 335.87it/s]


In [36]:
world.ShowEnvironments()

In [37]:
world.ShowFeatures()

In [38]:
world.real_reward_arr.shape

(78,)

In [39]:
world.ShowGridWorld_Activated()

In [43]:
dme = DMEIRL(world,layers = (16,32,64,32,16),lr = 0.0005,log=utils.date+'(4)',log_dir='test_run')
rewards_result1,rewards1 = dme.train(n_epochs=300,save = False,demo=True)

100%|██████████| 300/300 [10:57<00:00,  2.19s/it]


In [44]:
world.ShowRewardsResult(rewards_result1,title='DMEIRL奖励复原结果')

In [45]:
world.ShowRewardsAnimation(rewards1,title='DMEIRL奖励复原动画')

In [18]:
dme.SaveModel('1_model.pth')